In [1]:
%cd ../src

/home/ubuntu/SPVD_Lightning/src


In [ ]:
from utils.visualization import visualize_notebook
import torch

model_state_dict = torch.load("../checkpoints/ModelNet/testing/unsafe.ckpt", weights_only=True)['state_dict']
model.load_state_dict(model_state_dict)
model = model.cuda().eval()

from utils.schedulers import DDPMSparseSchedulerGPU

ddpm_sched = DDPMSparseSchedulerGPU(n_steps=1000, beta_min=0.0001, beta_max=0.02, pres=1e-5)

In [2]:
from my_models.spvd import SPVUnet

down_blocks = [{
    "features_list": [32, 64, 128, 192, 192, 256],
    "num_layers_list": 1,
    "attn_heads": (None, None, None, 8, 8)
}]
up_blocks = [
    {
        "features_list": [256, 192, 192],
        "num_layers_list": 2,
        "attn_heads": 8
    },
    {
        "features_list": [192, 128, 64, 32],
        "num_layers_list": 2,
        "attn_heads": None
    },
]
t_emb_features = 64

model = SPVUnet(down_blocks, up_blocks, t_emb_features)

import models

model = models.DiffusionBase(model, lr=1e-4)

/opt/conda/envs/spvd/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [ ]:
preds = ddpm_sched.sample(model, 16, 2048)
visualize_notebook(preds, x_offset=2.5, y_offset=2.5, point_size=0.025)

In [2]:
import models
import torch

m = models.SPVD()

lr = 1e-4
model2 = models.DiffusionBase(m, lr=lr)

# weights = torch.load("../checkpoints/ModelNet40/airplane.ckpt", weights_only=True)

# model2.load_state_dict(weights['state_dict'])

/opt/conda/envs/spvd/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [10]:
model2.model.summary()

Point Resolution: 1e-05 | Voxel Resolution: 0.1
Time Embedding size - before mlp: 32 , -after mlp: 128 
** StemConv ** 
 -- Voxel Branch 
 Conv3d(3,32) - Params: 30304 
 -- Point Branch 
 PointBlock(3,32) - Params: 134 
 -- Feature Mix : mixed
** Down Stage 0 ** 
 -- Voxel Branch 
 DownBlock(32, 64, add_down=True, num_layers=1, attn_chans=None) - Params: 80128 
 DownBlock(64, 128, add_down=True, num_layers=1, attn_chans=None) - Params: 303616 
 DownBlock(128, 192, add_down=True, num_layers=1, attn_chans=None) - Params: 1115136 
 DownBlock(192, 192, add_down=True, num_layers=1, attn_chans=8) - Params: 2484288 
 DownBlock(192, 256, add_down=False, num_layers=1, attn_chans=8) - Params: 2238528 
 -- Point Branch 
 PointBlock(32,256) - Params: 8512 
 -- Feature Mix : mixed
** Up Stage 0 ** 
 -- Voxel Branch 
 UpBlock(256, 192 | 192), add_up=True, num_layers = 2, attn_chans=8 - Params: 11016960 
 UpBlock(192, 192 | 192), add_up=True, num_layers = 2, attn_chans=8 - Params: 6812928 
 -- Point 

In [3]:
from utils.visualization import visualize_notebook
from utils.schedulers import DDPMSparseSchedulerGPU

ddpm_sched = DDPMSparseSchedulerGPU(n_steps=1000, beta_min=0.0001, beta_max=0.02, pres=1e-5)

preds = ddpm_sched.sample(model2.cuda().eval(), 16, 2048)
print(preds.shape)

  0%|▌                                                                                                                                                                                       | 3/1000 [00:00<02:55,  5.69it/s]

10
6
0
10
6
0
10
6
0


  0%|▉                                                                                                                                                                                       | 5/1000 [00:00<02:00,  8.26it/s]

10
6
0
10
6
0
10
6
0


  1%|█▋                                                                                                                                                                                      | 9/1000 [00:01<01:27, 11.26it/s]

...

  4%|███████▌                                                                                                                                                                               | 41/1000 [00:03<01:37,  9.81it/s]

10
6
0
10
6
0


  4%|███████▊                                                                                                                                                                               | 43/1000 [00:04<01:31, 10.48it/s]

10
6
0
10
6
0


KeyboardInterrupt: 

In [4]:
visualize_notebook(preds, x_offset=2.5, y_offset=2.5, point_size=0.05)

Output()

In [23]:
from datasets.modelnet40.modelnet40_loader import get_dataloaders, ModelNet40

path = "../data/ModelNet40"
categories = ['airplane']

tr, te = get_dataloaders(path, categories=categories)

Loading renders for airplane: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 7291.27it/s]


In [30]:
t = ModelNet40(path=path, categories=categories)

preds = torch.stack([t[i]['pc'] for i in range(16)])[0].unsqueeze(0)

print(preds.shape)

visualize_notebook(preds, x_offset=2.5, y_offset=2.5, point_size=0.05)

Loading renders for airplane: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 626/626 [00:00<00:00, 7517.73it/s]

torch.Size([1, 5000, 3])


Output()